In [1]:
import requests
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from nba_api.stats.endpoints import leaguedashplayershotlocations

In [2]:
leaguedashplayershotlocations.LeagueDashPlayerShotLocations


nba_api.stats.endpoints.leaguedashplayershotlocations.LeagueDashPlayerShotLocations

In [3]:
def shot_location_scraper(start_yr, end_yr):
    for yr in range(int(start_yr), int(end_yr)+1):
        yr_str = '{}-{}'.format(yr, str(yr+1)[-2:])
        %time player_info = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(season=yr_str)
        yr_data = player_info.get_dict()['resultSets']
        headers = yr_data['headers']
        data = yr_data['rowSet']
        sub_headers = headers[1]['columnNames']
        top_headers = headers[0]['columnNames']
        top_headers_name = headers[0]['name']
        sub_headers_name = headers[1]['name']
        cur_heads = pd.MultiIndex.from_product([['Player_Info'], sub_headers[:5]])
        final_headers = cur_heads.append(pd.MultiIndex.from_product([top_headers, sub_headers[5:8]], 
                                                  names=[top_headers_name, sub_headers_name]))
        pd.DataFrame(data=data, columns=final_headers).to_csv('data/{}_shot_locations.csv'.format(yr_str), index=False)
        print('{} done'.format(yr_str))

In [4]:
#shot_location_scraper(2000, 2020)

In [5]:
link = requests.get("https://stats.nba.com/teams/traditional/?sort=FG3M&dir=-1&Season=2019-20&SeasonType=Regular%20Season&PerMode=Totals")

In [6]:
html = link.content
parser = BeautifulSoup(html, "html.parser")

In [7]:
all_div = parser.find("tbody")

In [8]:
all_div

In [11]:
bs = BeautifulSoup(html)
table = bs.find(lambda tag: tag.name=='div' and tag.has_attr('class') and tag['class']=="stats-table-pagination") 
print(table)
#rows = table.findAll(lambda tag: tag.name=='tr')

None


In [15]:
all_stats = pd.read_csv("NBA_Stats-Sheet1.csv")
all_stats.head()

,Unnamed: 0,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,Season
0,1,Milwaukee Bucks,65,53,12,0.815,48.2,118.6,43.5,91.2,...,42.2,51.7,25.9,14.9,7.4,6.0,4.6,19.2,21.3,2019-20
1,2,Los Angeles Lakers,63,49,14,0.778,48.2,114.3,42.9,88.6,...,35.5,46.1,25.9,15.1,8.6,6.8,3.7,20.6,21.4,2019-20
2,3,Toronto Raptors,64,46,18,0.719,48.3,113.0,40.6,88.5,...,35.5,45.2,25.4,14.4,8.8,4.9,5.3,21.5,20.0,2019-20
3,4,LA Clippers,64,44,20,0.688,48.2,116.2,41.6,89.7,...,37.0,48.0,23.8,14.8,7.1,5.0,4.9,22.0,22.8,2019-20
4,5,Boston Celtics,64,43,21,0.672,48.4,113.0,41.2,89.6,...,35.3,46.0,22.8,13.6,8.3,5.6,5.6,21.4,20.6,2019-20


In [16]:
all_stats.columns
#3:00 Pm is 3 pointers made/game

Index(['Unnamed: 0', 'TEAM', 'GP', 'W', 'L', 'WIN%', 'MIN', 'PTS', 'FGM',
       'FGA', 'FG%', '3:00 PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB',
       'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD',
       'Season'],
      dtype='object')

In [17]:
all_stats.groupby("Season").mean()["PTS"]

Season
2000-01     94.810345
2001-02     95.472414
2002-03     95.089655
2003-04     93.396552
2004-05     97.190000
2005-06     97.013333
2006-07     98.740000
2007-08     99.923333
2008-09     99.943333
2009-10    100.450000
2010-11     99.553333
2011-12     96.260000
2012-13     98.136667
2013-14    101.010000
2014-15    100.016667
2015-16    102.663333
2016-17    105.590000
2017-18    106.336667
2018-19    111.203333
2019-20    111.463333
Name: PTS, dtype: float64

In [18]:
all_stats.groupby("Season").mean()["3:00 PM"]

Season
2000-01     4.841379
2001-02     5.213793
2002-03     5.134483
2003-04     5.182759
2004-05     5.600000
2005-06     5.726667
2006-07     6.070000
2007-08     6.563333
2008-09     6.653333
2009-10     6.440000
2010-11     6.463333
2011-12     6.406667
2012-13     7.163333
2013-14     7.743333
2014-15     7.850000
2015-16     8.506667
2016-17     9.650000
2017-18    10.486667
2018-19    11.360000
2019-20    12.140000
Name: 3:00 PM, dtype: float64

In [19]:
Two_pointers_pg = (all_stats.groupby("Season").mean()["PTS"] - all_stats.groupby("Season").mean()["3PA"])/2

In [20]:
Two_pointers_pg

Season
2000-01    40.551724
2001-02    40.358621
2002-03    40.206897
2003-04    39.237931
2004-05    40.720000
2005-06    40.515000
2006-07    40.900000
2007-08    40.908333
2008-09    40.905000
2009-10    41.156667
2010-11    40.771667
2011-12    38.936667
2012-13    39.085000
2013-14    39.736667
2014-15    38.803333
2015-16    39.290000
2016-17    39.293333
2017-18    38.670000
2018-19    39.598333
2019-20    38.761667
dtype: float64